In [1]:
import numpy as np
import numpy.linalg as lg
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
G=nx.read_weighted_edgelist('Tortoise.txt') # Load in a file
A=nx.to_numpy_array(G)

In [3]:
G.adjacency

<bound method Graph.adjacency of <networkx.classes.graph.Graph object at 0x7f0cb3600b50>>

In [4]:
def freqG(A):
    n = len(A) # nodes
    e = np.ones(n)
    k = A@e # degrees.
    m = np.sum(k)/2 # edges
    Lambda, Q = lg.eigh(A)
    # A number of matrices/vectors we'll need in our computations
    A2 = A@A 
    A3 = A2@A
    A2a = A2*A
    A2b = A2 - np.diag(k)
    F10s = sum(A2b*(A2b-1))/2
    t = np.diag(A3)/2
    tk = t*(k-2)
    kk = k*(k-1)
    # The fragments F1...F15
    P2 = sum(kk/2)
    C3 = sum(t)/3
    P3 = (k-1).T@A@(k-1)/2-C3*3 
    S13 = kk.T@(k-2)/6
    C4 = (round(np.sum(Lambda**4)) - 4*P2 - 2*m)/8
    T31 = sum(tk)
    Diamond = sum(sum((A2a)*(A2a-1)))/4
    C5 = (round(np.sum(Lambda**5)) - 30*C3 - 10*T31)/10 # Number of C5
    Cr = tk.T@(k-3)/2
    T41 = (k-2).T@F10s - 2*Diamond
    Reindeer = (k-2).T@(A2*A)@(k-2)/2 - 2*Diamond
    T32 = t.T@sum(A2b) - 6*C3 - 2*T31 - 4*Diamond
    Bowtie = t.T@(t-1)/2-2*Diamond
    House = sum(sum(A*A2*A3))/2-9*C3-2*T31-4*Diamond
    C6 = (round(np.sum(Lambda**6)) - 2*m - 12*P2 -24*C3 - 6*P3 - 12*S13 - 48*C4 - 36*Diamond - 12*T41 - 24*Bowtie)/12
    Count = np.array([P2, C3, P3, S13, C4, T31, Diamond, C5, Cr, T41, Reindeer, T32, Bowtie, House, C6])
    return Count

In [5]:
Count = freqG(A)
print(Count)

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [12]:
Trials = 50 # Average over at least 50 random graphs
CountR = np.zeros(15)
n = len(A) # nodes
e = np.ones(n)
k = A@e # degrees.
m = np.sum(k)/2 # edges
for i in range(Trials):
    R=nx.erdos_renyi_graph(n,2*m/n/(n-1))  # Here we create a random graph
    AR=nx.to_numpy_array(R)    # The rest of the loop just makes use of the formulae calculate earlier
    f = freqG(AR)
    CountR = CountR + f
CountR = CountR/Trials


In [13]:
abund=(Count-CountR)/(Count+CountR)
print(abund)
#If abund >.7 then we can call it a motif. If abund <-.5 it's an anti-motif

[0.11506248 0.46118721 0.21129047 0.29413873 0.45629287 0.56161694
 0.77537932 0.47267299 0.6780824  0.55125429 0.63813887 0.60333418
 0.80501798 0.75072023 0.48897196]


In [14]:
print(Count)

[  671.    96.  4551.  1805.   397.  2100.   454.  1766.  8151. 10975.
 14303. 12826.  1441.  4424.  7966.]


In [7]:
!pip install seaborn

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.11/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [8]:
# ==============================================================================
# Network Analysis Script for 'Tortoise.txt'
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. IMPORTS
# ------------------------------------------------------------------------------
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
import numpy.linalg as lg # For matrix operations if needed, though NetworkX handles most
import seaborn as sns
from collections import Counter
import community as community_louvain # For Louvain community detection
import os # For file checking

print("--- Libraries Loaded ---")

# ------------------------------------------------------------------------------
# 2. GLOBAL SETTINGS AND HELPER FUNCTIONS
# ------------------------------------------------------------------------------
# Matplotlib styles
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 9) # Default figure size, larger for better visibility
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['figure.dpi'] = 100 # Good resolution for saved images

# Function to save plots if needed
SAVE_PLOTS = False # Set to True to save plots
PLOT_DIR = "network_analysis_plots"
if SAVE_PLOTS and not os.path.exists(PLOT_DIR):
    os.makedirs(PLOT_DIR)

def save_current_plot(filename):
    if SAVE_PLOTS:
        filepath = os.path.join(PLOT_DIR, filename)
        plt.savefig(filepath, bbox_inches='tight', dpi=150)
        print(f"Plot saved to {filepath}")

print("--- Global Settings Initialized ---")

# ------------------------------------------------------------------------------
# 3. LOAD DATA
# ------------------------------------------------------------------------------
TORTOISE_FILE = "Tortoise.txt"

# Create a dummy Tortoise.txt file for demonstration if it doesn't exist
if not os.path.exists(TORTOISE_FILE):
    print(f"'{TORTOISE_FILE}' not found. Creating a dummy file for demonstration.")
    tortoise_file_content = """
0	1	1
0	2	1
1	2	1
1	3	1
2	3	1
0	4	1
4	5	1
0	6	1
4	6	1
0	7	1
0	8	1
5	8	1
6	8	1
7	8	1
0	9	1
2	9	1
3	9	1
4	9	1
6	9	1
8	9	1
0	10	1
1	10	1
2	10	1
3	10	1
9	10	1
0	11	1
5	11	1
6	11	1
1	12	1
3	12	1
6	12	1
10	12	1
11	12	1
1	13	1
12	13	1
1	14	1
2	14	1
3	14	1
6	14	1
9	14	1
10	14	1
11	15	1
12	15	1
5	16	1
8	16	1
11	16	1
3	17	1
5	17	1
6	17	1
8	17	1
9	17	1
10	17	1
11	17	1
12	17	1
14	17	1
15	17	1
3	18	1
5	18	1
6	18	1
8	18	1
11	18	1
15	18	1
17	18	1
3	19	1
6	19	1
9	19	1
2	20	1
6	20	1
8	20	1
10	20	1
17	20	1
18	20	1
6	21	1
17	21	1
18	21	1
9	23	1
22	23	1
22	24	1
23	25	1
24	25	1
6	26	1
8	26	1
9	26	1
19	26	1
19	27	1
26	27	1
27	28	1
6	29	1
9	29	1
19	29	1
26	29	1
27	29	1
6	30	1
"""
    with open(TORTOISE_FILE, "w") as f:
        f.write(tortoise_file_content.strip())

# Load the graph using read_weighted_edgelist
# It assumes space or tab as delimiter and that nodes are strings.
# We need to specify nodetype=int if nodes are integers.
try:
    G = nx.read_weighted_edgelist(TORTOISE_FILE, nodetype=int)
    print(f"\nGraph loaded successfully from '{TORTOISE_FILE}'")
    print(f"Number of nodes: {G.number_of_nodes()}")
    print(f"Number of edges: {G.number_of_edges()}")

    # Convert to NumPy array (Adjacency Matrix)
    # Note: Node labels in G might not be 0-indexed contiguously.
    # to_numpy_array handles this by using G.nodes() order or a specified nodelist.
    # For simplicity, let's assume nodes are 0 to N-1 or NetworkX handles it.
    node_list_for_A = sorted(list(G.nodes())) # Ensure consistent order for A
    A = nx.to_numpy_array(G, nodelist=node_list_for_A)
    print(f"Adjacency matrix A created with shape: {A.shape}")

    # Display a few edges from G
    print("\nSample Edges from G (first 5):")
    for i, (u, v, data) in enumerate(list(G.edges(data=True))[:5]):
        print(f"({u}, {v}, weight: {data.get('weight', 1)})")

    # Initial plot of the network
    plt.figure(figsize=(12, 12)) # Larger size for initial plot
    pos_spring = nx.spring_layout(G, seed=42, k=0.7, iterations=50) # k adjusts spacing, iterations for convergence
    nx.draw_networkx_nodes(G, pos_spring, node_size=200, node_color='skyblue', alpha=0.9, edgecolors='black', linewidths=0.5)
    nx.draw_networkx_edges(G, pos_spring, width=1.2, alpha=0.4, edge_color='gray')
    nx.draw_networkx_labels(G, pos_spring, font_size=9, font_weight='bold')
    plt.title("Initial Visualization of the Tortoise Network", fontsize=20)
    plt.axis('off')
    plt.tight_layout()
    save_current_plot("00_initial_network_plot.png")
    plt.show()

except FileNotFoundError:
    print(f"ERROR: The file '{TORTOISE_FILE}' was not found. Please ensure it's in the correct directory.")
    exit()
except Exception as e:
    print(f"An error occurred during data loading or initial plotting: {e}")
    exit()

print("\n--- Data Loading and Initial Plot Complete ---")

# ------------------------------------------------------------------------------
# 4. DATA DESCRIPTION AND PREVIOUS RESEARCH (TEXTUAL - TO BE FILLED BY USER)
# ------------------------------------------------------------------------------
print("""
# ==============================================================================
# 4. Data Description and Previous Research
# ==============================================================================

# **Origin of the Data:**
# The dataset is named `Tortoise.txt`. 
# (USER: Fill this part. Where did this dataset come from? What real-world system does it represent? 
# E.g., "This dataset represents social interactions observed among a population of Gopher tortoises 
# (Gopherus polyphemus) at a specific research site over a defined period.")

# **Nodes and Edges Representation:**
# *   **Nodes:** What do the integers (0, 1, 2, ...) represent? 
#     (USER: e.g., individual tortoises, specific locations, etc.)
# *   **Edges:** What does a connection (edge) between two nodes signify? 
#     (USER: e.g., observed interaction, shared burrow, genetic relatedness, etc.) 
#     The edges in the file have a third column, which `nx.read_weighted_edgelist`
#     interprets as a weight. In the provided example, all weights are '1', 
#     suggesting either an unweighted graph or that all interactions have a uniform weight/strength.

# **Previous Research on the Network:**
# (USER: This is a crucial part for your assignment. You need to search for literature 
# related to this specific dataset or similar types of networks. For example, if it's an 
# animal social network, you would look for papers on:)
# *   *Studies using this specific dataset (if it's known).*
# *   *General studies on social network analysis in [the specific animal species or context].*
# *   *Commonly observed network structures or properties in such systems.*
# *   *How network analysis has provided insights into the behavior, disease transmission, 
#      or conservation of the entities represented by the nodes.*

# **Example (if it were a known dataset like Zachary's Karate Club):**
# "The Zachary's Karate Club network is a well-studied dataset representing friendships 
# between 34 members of a karate club at a US university in the 1970s. The network 
# famously split into two factions after a dispute. Research on this network often 
# focuses on community detection algorithms and predicting this split."

# **For the Tortoise dataset, you might look for keywords like:**
# *   "Tortoise social network"
# *   "Animal interaction network analysis"
# *   "[Species name if known] network structure"

# (USER: Without more context on the `Tortoise.txt` origin, this section remains a template for you to fill.)
""")
print("\n--- Data Description Section (Placeholder) Printed ---")

# ------------------------------------------------------------------------------
# 5. BASIC STATISTICS OF YOUR NETWORK
# ------------------------------------------------------------------------------
print("\n# ============================================================================== ")
print("# 5. Basic Statistics of Your Network ")
print("# ============================================================================== ")

num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
print(f"Number of nodes (N): {num_nodes}")
print(f"Number of edges (M): {num_edges}")

if num_nodes == 0:
    print("Graph is empty. Cannot compute further statistics.")
    exit()

density = nx.density(G)
print(f"Density of the graph: {density:.4f}")

degrees = [d for n, d in G.degree()]
avg_degree = np.mean(degrees) if degrees else 0
print(f"Average degree: {avg_degree:.2f}")
print(f"Minimum degree: {np.min(degrees) if degrees else 0}")
print(f"Maximum degree: {np.max(degrees) if degrees else 0}")

is_connected_graph = nx.is_connected(G)
print(f"Is the graph connected? {is_connected_graph}")

num_components = 0
avg_path_length_lcc = float('nan')
diameter_lcc = float('nan')

if not is_connected_graph:
    num_components = nx.number_connected_components(G)
    print(f"Number of connected components: {num_components}")
    largest_cc_nodes = max(nx.connected_components(G), key=len)
    LCC = G.subgraph(largest_cc_nodes).copy()
    print(f"Size of largest connected component (LCC): {LCC.number_of_nodes()} nodes, {LCC.number_of_edges()} edges")
    if LCC.number_of_nodes() > 1:
        try:
            avg_path_length_lcc = nx.average_shortest_path_length(LCC)
            diameter_lcc = nx.diameter(LCC)
        except nx.NetworkXError as e:
            print(f"Could not compute path metrics for LCC: {e}")
    else:
        print("LCC is too small for path metrics.")
else:
    LCC = G
    print("The graph is connected, so LCC is the graph itself.")
    if LCC.number_of_nodes() > 1:
        try:
            avg_path_length_lcc = nx.average_shortest_path_length(LCC)
            diameter_lcc = nx.diameter(LCC)
        except nx.NetworkXError as e:
            print(f"Could not compute path metrics for LCC: {e}")
    else:
        print("Graph is too small for path metrics.")
        
print(f"Average shortest path length (for LCC): {avg_path_length_lcc:.2f}" if not np.isnan(avg_path_length_lcc) else "Avg. Path Length (LCC): N/A")
print(f"Diameter (for LCC): {diameter_lcc}" if not np.isnan(diameter_lcc) else "Diameter (LCC): N/A")

avg_clustering_coeff = nx.average_clustering(G)
print(f"Average clustering coefficient (global): {avg_clustering_coeff:.4f}")

transitivity = nx.transitivity(G)
print(f"Transitivity (global CC variant): {transitivity:.4f}")

is_bipartite_graph = nx.is_bipartite(G)
print(f"Is the graph bipartite? {is_bipartite_graph}")
if is_bipartite_graph:
    bipartite_sets = nx.bipartite.sets(G)
    print(f"Bipartite sets (U, V): (Size {len(bipartite_sets[0])}, Size {len(bipartite_sets[1])})")

assortativity = nx.degree_assortativity_coefficient(G)
print(f"Degree assortativity coefficient: {assortativity:.4f}")

# Degree Distribution Plot
plt.figure(figsize=(10, 7))
sns.histplot(degrees, bins=max(1, int(np.sqrt(len(degrees)))), kde=True, color='dodgerblue', ec='black')
plt.title('Degree Distribution of the Tortoise Network', fontsize=18)
plt.xlabel('Degree', fontsize=14)
plt.ylabel('Frequency of Nodes', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
save_current_plot("01_degree_distribution.png")
plt.show()

stats_summary_df = pd.DataFrame({
    'Metric': ['Nodes', 'Edges', 'Density', 'Average Degree', 'Min Degree', 'Max Degree', 'Connected?', 
               'Num. Components (if not connected)', 'Avg. Path Length (LCC)', 'Diameter (LCC)',
               'Avg. Clustering Coeff.', 'Transitivity', 'Bipartite?', 'Assortativity'],
    'Value': [num_nodes, num_edges, f"{density:.4f}", f"{avg_degree:.2f}", np.min(degrees) if degrees else 0, np.max(degrees) if degrees else 0, is_connected_graph,
              num_components if not is_connected_graph else 'N/A', 
              f"{avg_path_length_lcc:.2f}" if not np.isnan(avg_path_length_lcc) else 'N/A',
              diameter_lcc if not np.isnan(diameter_lcc) else 'N/A',
              f"{avg_clustering_coeff:.4f}", f"{transitivity:.4f}", is_bipartite_graph, f"{assortativity:.4f}"]
})
print("\n--- Summary Table of Basic Statistics ---")
print(stats_summary_df.to_string(index=False))
print("\n--- Basic Statistics Calculation Complete ---")

# ------------------------------------------------------------------------------
# 6. CENTRALITY MEASURES (Illustrations of the five nodes of highest degree)
# ------------------------------------------------------------------------------
print("\n# ============================================================================== ")
print("# 6. Centrality Measures ")
print("# ============================================================================== ")

# 1. Degree Centrality (Classical)
degree_centrality = nx.degree_centrality(G)
sorted_degree_cent = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)
top5_degree = sorted_degree_cent[:5]

# 2. Betweenness Centrality (Classical)
betweenness_centrality = nx.betweenness_centrality(G, weight='weight') # Consider weights if meaningful
sorted_betweenness_cent = sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True)
top5_betweenness = sorted_betweenness_cent[:5]

# 3. Eigenvector Centrality (Spectral)
# For Eigenvector centrality, graph must be connected for standard algorithm.
# NetworkX handles disconnected graphs by computing for the largest component,
# or returning 0 for nodes not in a component allowing eigenvector calculation.
# It might fail to converge for some graphs.
centrality_measure_name_3 = "Eigenvector Centrality (Spectral)"
try:
    eigenvector_centrality = nx.eigenvector_centrality_numpy(G, weight='weight') # Using numpy version for stability
    sorted_eigenvector_cent = sorted(eigenvector_centrality.items(), key=lambda item: item[1], reverse=True)
    top5_eigenvector = sorted_eigenvector_cent[:5]
except (nx.NetworkXError, nx.PowerIterationFailedConvergence) as e:
    print(f"Eigenvector centrality failed: {e}. Using PageRank as an alternative spectral measure.")
    pagerank_centrality = nx.pagerank(G, weight='weight')
    sorted_pagerank_cent = sorted(pagerank_centrality.items(), key=lambda item: item[1], reverse=True)
    top5_eigenvector = sorted_pagerank_cent[:5] # Using the same variable name for top5 for consistency
    eigenvector_centrality = pagerank_centrality # Use PageRank values
    centrality_measure_name_3 = "PageRank Centrality (Spectral Alt.)"

# 4. Closeness Centrality (Classical)
closeness_centrality = nx.closeness_centrality(G, distance='weight') # distance if weights are path lengths
sorted_closeness_cent = sorted(closeness_centrality.items(), key=lambda item: item[1], reverse=True)
top5_closeness = sorted_closeness_cent[:5]

print("--- Top 5 Nodes by Centrality Measures ---")
print(f"Top 5 by Degree Centrality: {top5_degree}")
print(f"Top 5 by Betweenness Centrality: {top5_betweenness}")
print(f"Top 5 by {centrality_measure_name_3}: {top5_eigenvector}")
print(f"Top 5 by Closeness Centrality: {top5_closeness}")

# Create a DataFrame for easier comparison
centrality_data = {
    'Node': list(G.nodes()),
    'Degree_Cent': [degree_centrality.get(n, 0) for n in G.nodes()],
    'Betweenness_Cent': [betweenness_centrality.get(n, 0) for n in G.nodes()],
    centrality_measure_name_3.split(' ')[0] + '_Cent': [eigenvector_centrality.get(n, 0) for n in G.nodes()],
    'Closeness_Cent': [closeness_centrality.get(n, 0) for n in G.nodes()]
}
centrality_df = pd.DataFrame(centrality_data).set_index('Node').sort_values(by='Degree_Cent', ascending=False)
print("\n--- Centrality Values for Top Nodes (Sorted by Degree Centrality) ---")
print(centrality_df.head(10).to_string(float_format="%.4f"))

# Plotting Centrality Distributions
fig_cent_dist, axes_cent_dist = plt.subplots(2, 2, figsize=(18, 14))
fig_cent_dist.suptitle('Distributions of Centrality Measures', fontsize=22, y=0.98)

sns.histplot(centrality_df['Degree_Cent'], bins=15, kde=True, ax=axes_cent_dist[0, 0], color='cornflowerblue', ec='black')
axes_cent_dist[0, 0].set_title('Degree Centrality')
sns.histplot(centrality_df['Betweenness_Cent'], bins=15, kde=True, ax=axes_cent_dist[0, 1], color='salmon', ec='black')
axes_cent_dist[0, 1].set_title('Betweenness Centrality')
if centrality_df['Betweenness_Cent'].max() > 0 and centrality_df['Betweenness_Cent'].min() >= 0 and centrality_df['Betweenness_Cent'].skew() > 2:
    axes_cent_dist[0, 1].set_yscale('log')
    axes_cent_dist[0, 1].set_ylabel('Frequency (Log Scale)')


sns.histplot(centrality_df[centrality_measure_name_3.split(' ')[0] + '_Cent'], bins=15, kde=True, ax=axes_cent_dist[1, 0], color='mediumseagreen', ec='black')
axes_cent_dist[1, 0].set_title(centrality_measure_name_3)
sns.histplot(centrality_df['Closeness_Cent'], bins=15, kde=True, ax=axes_cent_dist[1, 1], color='gold', ec='black')
axes_cent_dist[1, 1].set_title('Closeness Centrality')

for ax_row in axes_cent_dist:
    for ax in ax_row:
        ax.set_xlabel('Centrality Value')
        if not (ax == axes_cent_dist[0,1] and centrality_df['Betweenness_Cent'].skew() > 2) : # Avoid double labeling if log scale
             ax.set_ylabel('Frequency')
        ax.grid(True, linestyle=':', alpha=0.6)
plt.tight_layout(rect=[0, 0, 1, 0.95])
save_current_plot("02_centrality_distributions.png")
plt.show()

# Visualization of Top Nodes in the Network (Nodes colored by centrality score)
fig_cent_color, axes_cent_color = plt.subplots(2, 2, figsize=(22, 20)) # Larger figure
fig_cent_color.suptitle('Network: Nodes Colored by Centrality Score', fontsize=24, y=0.98)
all_nodes_list_sorted = sorted(list(G.nodes())) # Consistent node ordering

# Helper lists for labeling specific top nodes
top5_degree_nodes_only = [n for n,_ in top5_degree]
top5_betweenness_nodes_only = [n for n,_ in top5_betweenness]
top5_eigenvector_nodes_only = [n for n,_ in top5_eigenvector]
top5_closeness_nodes_only = [n for n,_ in top5_closeness]
all_top_nodes = set(top5_degree_nodes_only + top5_betweenness_nodes_only + top5_eigenvector_nodes_only + top5_closeness_nodes_only)

# Function to draw a centrality-colored graph
def draw_centrality_graph(ax, G_graph, pos_layout, centrality_values_dict, cmap_color, title, label_nodes, centrality_label):
    node_values = [centrality_values_dict.get(node, 0) for node in all_nodes_list_sorted]
    max_val = max(node_values) if node_values else 1
    min_val = min(node_values) if node_values else 0
    
    node_sizes = [val * 5000 / (max_val if max_val >0 else 1) + 100 for val in node_values] # Dynamic sizing
    if not node_values or max_val == 0: # Handle cases with all zero centrality
        node_sizes = [150 for _ in node_values]

    nx.draw_networkx_nodes(G_graph, pos_layout, nodelist=all_nodes_list_sorted, node_color=node_values, 
                           cmap=cmap_color, node_size=node_sizes, ax=ax, alpha=0.85, edgecolors='grey', linewidths=0.5)
    nx.draw_networkx_edges(G_graph, pos_layout, ax=ax, alpha=0.25, edge_color='darkgray', width=1.0)
    nx.draw_networkx_labels(G_graph, pos_layout, ax=ax, font_size=9, font_weight='bold',
                            labels={n:n for n in label_nodes if n in G_graph}) # Label only specified nodes
    
    ax.set_title(title, fontsize=18)
    ax.axis('off')
    # Colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap_color, norm=plt.Normalize(vmin=min_val, vmax=max_val if max_val > min_val else min_val + 1e-9)) # Avoid vmin=vmax
    sm.set_array([])
    cbar = fig_cent_color.colorbar(sm, ax=ax, orientation='vertical', fraction=0.046, pad=0.04, shrink=0.8)
    cbar.set_label(centrality_label, fontsize=12)

draw_centrality_graph(axes_cent_color[0,0], G, pos_spring, degree_centrality, plt.cm.viridis, 'Degree Centrality', top5_degree_nodes_only, 'Degree Centrality')
draw_centrality_graph(axes_cent_color[0,1], G, pos_spring, betweenness_centrality, plt.cm.plasma, 'Betweenness Centrality', top5_betweenness_nodes_only, 'Betweenness Centrality')
draw_centrality_graph(axes_cent_color[1,0], G, pos_spring, eigenvector_centrality, plt.cm.cividis, centrality_measure_name_3, top5_eigenvector_nodes_only, centrality_measure_name_3)
draw_centrality_graph(axes_cent_color[1,1], G, pos_spring, closeness_centrality, plt.cm.magma, 'Closeness Centrality', top5_closeness_nodes_only, 'Closeness Centrality')

plt.tight_layout(rect=[0, 0, 1, 0.96])
save_current_plot("03_centrality_colored_graphs.png")
plt.show()

print("--- Centrality Analysis and Visualization Complete ---")
print("""
--- Discussion on Centrality Similarities/Differences (USER: Adapt this based on your results!) ---
- Degree centrality identifies nodes with many direct connections (local influence).
- Betweenness centrality identifies "bridge" nodes crucial for flow between network parts.
- Eigenvector/PageRank centrality identifies nodes connected to other influential nodes (global influence).
- Closeness centrality identifies nodes that can reach others quickly on average.

Observe which nodes appear in multiple top-5 lists. For example:
  'Node X is high in both degree and betweenness, suggesting it's a local hub AND a critical connector.'
  'Node Y is high in eigenvector centrality but not necessarily degree, indicating it's connected to other influential nodes rather than just many nodes.'
  'Node Z might be high in closeness, making it efficient for spreading information, but not a major hub by degree.'
""")

# ------------------------------------------------------------------------------
# 7. RELATIVE ABUNDANCE OF SUBGRAPHS (MOTIFS)
# ------------------------------------------------------------------------------
print("\n# ============================================================================== ")
print("# 7. Relative Abundance of Subgraphs (Motifs) ")
print("# ============================================================================== ")
# The assignment mentions "you will be provided with some code to compute this".
# If specific code is provided, use that. Here, we count triangles.

num_triangles_per_node = nx.triangles(G)
total_triangles = sum(num_triangles_per_node.values()) // 3
print(f"Total number of triangles in the graph: {total_triangles}")

if total_triangles > 0:
    plt.figure(figsize=(10, 7))
    sns.histplot(list(num_triangles_per_node.values()), 
                 bins=max(1, max(num_triangles_per_node.values()) if num_triangles_per_node else 1), 
                 kde=False, color='teal', ec='black')
    plt.title('Distribution of Triangles per Node', fontsize=18)
    plt.xlabel('Number of Triangles Node is Part Of', fontsize=14)
    plt.ylabel('Frequency of Nodes', fontsize=14)
    if num_triangles_per_node and max(num_triangles_per_node.values()) > 20 : plt.yscale('log')
    plt.grid(True, linestyle=':', alpha=0.6)
    save_current_plot("04_triangle_distribution.png")
    plt.show()
else:
    print("No triangles found in the graph.")

print("""
--- Interpretation of Subgraph (Triangle) Analysis ---
Triangles are the simplest form of a cluster. A high number of triangles often correlates 
with a high clustering coefficient, indicating local dense connectivity. This can be compared 
to random graph models. If the real network has significantly more triangles than a random 
graph of similar size/density, it suggests non-random structuring (e.g., due to triadic closure).

(USER: If provided with specific code for other motifs, integrate it here and discuss 
over/under-representation compared to random networks.)
""")
print("\n--- Subgraph Analysis Complete ---")

# ------------------------------------------------------------------------------
# 8. COMMUNITY STRUCTURE WITHIN YOUR NETWORK
# ------------------------------------------------------------------------------
print("\n# ============================================================================== ")
print("# 8. Community Structure ")
print("# ============================================================================== ")

communities = None
partition = None
modularity_score = float('nan')
num_communities = 0

if G.number_of_nodes() == 0 or G.number_of_edges() == 0:
    print("Graph is empty or has no edges. Cannot perform community detection.")
else:
    try:
        partition = community_louvain.best_partition(G, weight='weight', random_state=42)
        modularity_score = community_louvain.modularity(partition, G, weight='weight')
        num_communities = len(set(partition.values()))
        
        print(f"Detected {num_communities} communities using the Louvain algorithm.")
        print(f"Modularity of the partition: {modularity_score:.4f} (Higher is better, typically 0.3-0.7 indicates good structure)")

        community_counts = Counter(partition.values())
        print("\nCommunity sizes (Top 10 largest):")
        for comm_id, count in sorted(community_counts.items(), key=lambda item: item[1], reverse=True)[:10]:
            print(f"  Community {comm_id}: {count} nodes")

        communities = [[] for _ in range(num_communities)]
        for node, comm_id in partition.items():
            communities[comm_id].append(node) # communities is a list of lists of nodes

    except Exception as e:
        print(f"Error during community detection: {e}")

if partition and num_communities > 0:
    plt.figure(figsize=(12, 7))
    comm_ids_sorted = sorted(community_counts, key=community_counts.get, reverse=True)
    sns.barplot(x=[str(c) for c in comm_ids_sorted], 
                y=[community_counts[c] for c in comm_ids_sorted],
                palette="viridis_r") # Use a nice color palette
    plt.title('Distribution of Community Sizes', fontsize=18)
    plt.xlabel('Community ID (Sorted by Size)', fontsize=14)
    plt.ylabel('Number of Nodes', fontsize=14)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    save_current_plot("05_community_size_distribution.png")
    plt.show()

    # Visualize the graph with nodes colored by community
    plt.figure(figsize=(18, 18)) # Very large figure for clear community plot
    pos_comm_viz = nx.spring_layout(G, k=0.8, iterations=70, seed=42, weight='weight') # Layout sensitive to communities

    # Generate a color for each community
    if num_communities <= 10:
        unique_colors = plt.cm.get_cmap('tab10', num_communities)
    elif num_communities <=20:
        unique_colors = plt.cm.get_cmap('tab20', num_communities)
    else: 
        # For many communities, tableau and CSS colors might not be enough or distinct enough in some colormaps
        # Using a perceptually uniform colormap like 'viridis' and sampling from it
        # Or generate a list of distinguishable colors
        hsv_tuples = [(x * 1.0 / num_communities, 0.7, 0.9) for x in range(num_communities)]
        unique_colors_list = [mcolors.hsv_to_rgb(x) for x in hsv_tuples]
        unique_colors = mcolors.ListedColormap(unique_colors_list)


    node_colors_community = [unique_colors(partition[node] % unique_colors.N) for node in G.nodes()] # Modulo N for safety with colormaps

    nx.draw_networkx_nodes(G, pos_comm_viz, node_color=node_colors_community, 
                           node_size=250, alpha=0.9, edgecolors='black', linewidths=0.5)
    
    edge_colors_community = []
    edge_widths_community = []
    for u, v in G.edges():
        if partition[u] == partition[v]: # Intra-community edge
            edge_colors_community.append(unique_colors(partition[u] % unique_colors.N))
            edge_widths_community.append(1.5) # Thicker for intra
        else: # Inter-community edge
            edge_colors_community.append('lightgrey') 
            edge_widths_community.append(0.5) # Thinner for inter
            
    nx.draw_networkx_edges(G, pos_comm_viz, width=edge_widths_community, alpha=0.6, edge_color=edge_colors_community)
    nx.draw_networkx_labels(G, pos_comm_viz, font_size=9, font_weight='bold')

    plt.title(f'Network Communities (Louvain Algorithm, Modularity: {modularity_score:.3f})', fontsize=22)
    plt.axis('off')
    
    if 1 < num_communities <= 10: # Manageable legend
        legend_handles_comm = [plt.Line2D([0], [0], marker='o', color='w', 
                                          label=f'Comm {i} ({community_counts[i]} nodes)',
                                          markersize=12, markerfacecolor=unique_colors(i % unique_colors.N))
                               for i in sorted(community_counts.keys())[:num_communities]] # Show for sorted comm IDs
        plt.legend(handles=legend_handles_comm, loc='best', title="Communities", fontsize=11, frameon=True, fancybox=True, shadow=True)
        
    plt.tight_layout()
    save_current_plot("06_network_communities_visualization.png")
    plt.show()

    # Detailed Community Statistics
    print("\n--- Detailed Community Statistics ---")
    community_details_list = []
    for i in range(num_communities):
        comm_nodes = [node for node, comm_id in partition.items() if comm_id == i]
        if not comm_nodes: continue

        subgraph = G.subgraph(comm_nodes)
        comm_density = nx.density(subgraph)
        internal_edges = subgraph.number_of_edges()
        
        external_edges = 0
        boundary_nodes = set()
        for node_in_comm in comm_nodes:
            for neighbor in G.neighbors(node_in_comm):
                if partition.get(neighbor) != i: # Use .get for safety if a neighbor somehow isn't in partition
                    external_edges += 1
                    boundary_nodes.add(node_in_comm)
        
        comm_avg_degree_internal = (2 * internal_edges) / len(comm_nodes) if len(comm_nodes) > 0 else 0
        
        details = {
            "Comm_ID": i, "Nodes": len(comm_nodes), "Internal_Edges": internal_edges,
            "Density": f"{comm_density:.3f}", "External_Edges_Count": external_edges, # Sum of edges leaving
            "Boundary_Nodes": len(boundary_nodes), "Avg_Internal_Deg": f"{comm_avg_degree_internal:.2f}"
        }
        community_details_list.append(details)

    community_details_df = pd.DataFrame(community_details_list).sort_values(by="Nodes", ascending=False)
    print(community_details_df.to_string(index=False))
    
    if not centrality_df.empty:
        centrality_df['Community'] = centrality_df.index.map(partition.get)
        # Ensure 'Community' column is not NaN before grouping for robustness
        community_centrality_summary = centrality_df.dropna(subset=['Community']).groupby('Community').mean()
        print("\n--- Average Centrality Scores per Community ---")
        print(community_centrality_summary.to_string(float_format="%.3f"))
        
        if not community_centrality_summary.empty:
            community_centrality_summary.plot(kind='bar', figsize=(15,8), colormap='Spectral', edgecolor='black')
            plt.title('Average Centrality Scores by Community', fontsize=18)
            plt.ylabel('Average Centrality Value', fontsize=14)
            plt.xlabel('Community ID', fontsize=14)
            plt.xticks(rotation=0)
            plt.legend(title='Centrality Type', bbox_to_anchor=(1.02, 1), loc='upper left')
            plt.tight_layout(rect=[0,0,0.85,1]) # Adjust for legend
            save_current_plot("07_avg_centrality_per_community.png")
            plt.show()
else:
    print("Skipping detailed community analysis as no communities were found or an error occurred.")
print("\n--- Community Detection and Analysis Complete ---")

# ------------------------------------------------------------------------------
# 9. ANOTHER ASPECT OF NETWORK ANALYSIS (Originality will be rewarded)
#    Choosing K-Core Decomposition for this example.
# ------------------------------------------------------------------------------
print("\n# ============================================================================== ")
print("# 9. Another Aspect of Network Analysis: K-Core Decomposition ")
print("# ============================================================================== ")

if G.number_of_nodes() > 0:
    core_numbers = nx.core_number(G)
    max_core = 0
    if core_numbers:
        max_core = max(core_numbers.values())
        print(f"Maximum k-core number (main coreness): {max_core}")

        core_counts = Counter(core_numbers.values())
        plt.figure(figsize=(10, 7))
        sns.barplot(x=list(core_counts.keys()), y=list(core_counts.values()),
                    order=sorted(core_counts.keys()), palette="coolwarm_r", ec='black')
        plt.title('Distribution of Node Core Numbers', fontsize=18)
        plt.xlabel('Core Number (k)', fontsize=14)
        plt.ylabel('Number of Nodes', fontsize=14)
        plt.grid(True, linestyle=':', alpha=0.6)
        save_current_plot("08_k_core_distribution.png")
        plt.show()

        if max_core > 0:
            # Visualize nodes colored by core number
            plt.figure(figsize=(15,15))
            node_colors_kcore = [plt.cm.coolwarm_r(core_numbers.get(node,0) / max_core if max_core > 0 else 0) for node in G.nodes()]
            nx.draw_networkx_nodes(G, pos_spring, node_color=node_colors_kcore, node_size=200, alpha=0.9, edgecolors='grey')
            nx.draw_networkx_edges(G, pos_spring, width=1.0, alpha=0.3, edge_color='darkgray')
            nx.draw_networkx_labels(G, pos_spring, font_size=9)
            
            plt.title(f'Network Nodes Colored by K-Core Number (Max Core: {max_core})', fontsize=20)
            # Add colorbar for k-core
            sm_kcore = plt.cm.ScalarMappable(cmap=plt.cm.coolwarm_r, norm=plt.Normalize(vmin=0, vmax=max_core))
            sm_kcore.set_array([])
            cbar_kcore = plt.colorbar(sm_kcore, ax=plt.gca(), orientation='vertical', fraction=0.046, pad=0.04, ticks=range(0,max_core+1), shrink=0.8)
            cbar_kcore.set_label('Core Number (k)', fontsize=12)
            plt.axis('off')
            save_current_plot("09_nodes_colored_by_kcore.png")
            plt.show()
            
            main_core_nodes = [node for node, c_num in core_numbers.items() if c_num == max_core]
            print(f"Nodes in the {max_core}-core (most central cohesive group): {sorted(main_core_nodes)}")
        else:
            print("No significant cores (max_core = 0). The graph might be very sparse or tree-like.")
    else:
        print("Could not compute core numbers (e.g., empty graph).")
else:
    print("Graph is empty, skipping K-core decomposition.")

print("""
--- Interpretation of K-Core Decomposition ---
K-core decomposition identifies hierarchically nested groups of nodes based on their minimum 
degree within a subgraph. The innermost core (highest k) represents the most densely and 
resiliently connected part of the network. Nodes with higher core numbers are more central 
in terms of network cohesion and are less likely to be disconnected if other nodes are removed.
This can reveal the backbone or nucleus of the network.
""")
print("\n--- K-Core Analysis Complete ---")

# ------------------------------------------------------------------------------
# 10. COMPARISON WITH RANDOM GRAPH MODELS
# ------------------------------------------------------------------------------
print("\n# ============================================================================== ")
print("# 10. Comparison with Random Graph Models ")
print("# ============================================================================== ")

if N == 0:
    print("Original graph is empty, cannot generate random models.")
else:
    # Erdos-Renyi G(N,M) model
    gnm = nx.gnm_random_graph(N, M, seed=42)
    gnm_avg_clust = nx.average_clustering(gnm)
    gnm_avg_path = nx.average_shortest_path_length(max(nx.connected_components(gnm), key=len) if not nx.is_connected(gnm) else gnm) if gnm.number_of_nodes() > 1 else float('nan')

    # Erdos-Renyi G(N,p) model
    p_er = density 
    gnp = nx.erdos_renyi_graph(N, p_er, seed=42)
    gnp_avg_clust = nx.average_clustering(gnp)
    gnp_avg_path = nx.average_shortest_path_length(max(nx.connected_components(gnp), key=len) if not nx.is_connected(gnp) else gnp) if gnp.number_of_nodes() > 1 else float('nan')

    # Configuration Model (CM)
    cm = None
    cm_avg_clust = float('nan')
    cm_avg_path = float('nan')
    if sum(degrees) % 2 == 0 and degrees: # Check for valid degree sequence
        try:
            cm = nx.configuration_model(degrees, seed=42)
            cm = nx.Graph(cm) # Remove parallel edges and self-loops
            cm.remove_edges_from(nx.selfloop_edges(cm))
            cm_avg_clust = nx.average_clustering(cm)
            if cm.number_of_nodes() > 0 and cm.number_of_edges() > 0: # Check if CM is not empty
                 cm_avg_path = nx.average_shortest_path_length(max(nx.connected_components(cm), key=len, default=set()) if not nx.is_connected(cm) else cm) if cm.number_of_nodes() > 1 else float('nan')
            else: # CM became empty after simplification
                cm_avg_path = float('nan')
        except nx.NetworkXError as e_cm:
            print(f"   Error generating/processing Configuration Model: {e_cm}")
    else:
        print("Configuration model cannot be generated (e.g. odd degree sum or empty degree list).")


    comparison_data = {
        'Metric': ['Avg. Clustering Coeff.', 'Avg. Shortest Path (LCC/Largest)'],
        'Real Network (G)': [f"{avg_clustering_coeff:.4f}", f"{avg_path_length_lcc:.2f}" if not np.isnan(avg_path_length_lcc) else "N/A"],
        'G(N,M) Model': [f"{gnm_avg_clust:.4f}", f"{gnm_avg_path:.2f}" if not np.isnan(gnm_avg_path) else "N/A"],
        'G(N,p) Model': [f"{gnp_avg_clust:.4f}", f"{gnp_avg_path:.2f}" if not np.isnan(gnp_avg_path) else "N/A"],
        'Config. Model (CM)': [f"{cm_avg_clust:.4f}" if cm and not np.isnan(cm_avg_clust) else "N/A", 
                               f"{cm_avg_path:.2f}" if cm and not np.isnan(cm_avg_path) else "N/A"]
    }
    comparison_df = pd.DataFrame(comparison_data)
    print("\n--- Random Model Comparison Summary Table ---")
    print(comparison_df.to_string(index=False))

    # Plot Degree Distributions for comparison
    plt.figure(figsize=(15, 10))
    sns.histplot(degrees, color="red", label=f'Real Network (Avg: {avg_degree:.2f})', kde=False, stat="density", element="step", fill=False, linewidth=2.5)
    sns.histplot([d for n, d in gnm.degree()], color="blue", label=f'G(N,M) Model (Avg: {np.mean([d for n,d in gnm.degree()]):.2f})', kde=False, stat="density", element="step", fill=False, linewidth=1.5, linestyle='--')
    sns.histplot([d for n, d in gnp.degree()], color="green", label=f'G(N,p) Model (Avg: {np.mean([d for n,d in gnp.degree()]):.2f})', kde=False, stat="density", element="step", fill=False, linewidth=1.5, linestyle=':')
    if cm:
        cm_degrees = [d for n,d in cm.degree()]
        sns.histplot(cm_degrees, color="purple", label=f'Config. Model (Avg: {np.mean(cm_degrees):.2f})', kde=False, stat="density", element="step", fill=False, linewidth=1.5, linestyle='-.')

    plt.title('Comparison of Degree Distributions with Random Models', fontsize=20)
    plt.xlabel('Degree', fontsize=16)
    plt.ylabel('Density (Normalized Frequency)', fontsize=16)
    plt.legend(fontsize=12, loc='upper right', frameon=True, fancybox=True, shadow=True)
    plt.grid(True, linestyle=':', alpha=0.6)
    # Consider log scales if distributions are very skewed
    # plt.xscale('log')
    # plt.yscale('log')
    save_current_plot("10_degree_dist_comparison_random_models.png")
    plt.show()

print("""
--- Discussion on Comparison with Random Models ---
- Clustering Coefficient: Real-world networks often have higher clustering than G(N,M) or G(N,p). 
  The Configuration Model, by preserving degrees, might get closer but still often underestimates it 
  if there's true community structure or triadic closure beyond degree effects.
  (USER: Compare your G's clustering coeff: {avg_clustering_coeff:.4f} with the models.)

- Average Path Length: Real networks often exhibit the "small-world" property (short average paths), 
  comparable to or slightly larger than random graphs.
  (USER: Compare your G's LCC path length: {avg_path_length_lcc:.2f} with the models.)

- Degree Distribution: Erdos-Renyi graphs typically have a Poisson-like degree distribution. 
  If the real network shows a power-law or heavy-tailed distribution, it will differ. The 
  Configuration Model should match the real network's degree distribution by design.
  (USER: Observe the plot and comment on similarities/differences.)
""".format(avg_clustering_coeff=avg_clustering_coeff, avg_path_length_lcc=avg_path_length_lcc if not np.isnan(avg_path_length_lcc) else 0))
print("\n--- Random Model Comparison Complete ---")


# ------------------------------------------------------------------------------
# 11. DASHBOARD / SUMMARY OF FINDINGS
# ------------------------------------------------------------------------------
print("\n# ============================================================================== ")
print("# 11. Dashboard / Summary of Findings ")
print("# ============================================================================== ")

fig_dashboard, axs_dashboard = plt.subplots(2, 2, figsize=(24, 18)) # Large dashboard
fig_dashboard.suptitle("Tortoise Network Analysis: Key Insights Dashboard", fontsize=28, y=0.98)

# Panel 1: Basic Stats & Network Plot with Communities
ax_dash1 = axs_dashboard[0,0]
if partition and num_communities > 0 and 'unique_colors' in locals() and 'pos_comm_viz' in locals():
    node_colors_comm_dash = [unique_colors(partition[node] % unique_colors.N) for node in G.nodes()]
    edge_colors_dash = [unique_colors(partition[u] % unique_colors.N) if partition[u] == partition[v] else 'lightgray' for u, v in G.edges()]
    nx.draw_networkx_nodes(G, pos_comm_viz, node_color=node_colors_comm_dash, node_size=100, alpha=0.8, ax=ax_dash1, edgecolors='grey')
    nx.draw_networkx_edges(G, pos_comm_viz, edge_color=edge_colors_dash, width=0.7, alpha=0.5, ax=ax_dash1)
    ax_dash1.set_title(f'Network Structure & Communities ({num_communities} found, Modularity {modularity_score:.3f})', fontsize=18)
else:
    nx.draw_networkx(G, pos_spring, node_size=100, alpha=0.8, ax=ax_dash1, with_labels=True, font_size=8, width=0.7, node_color='skyblue', edge_color='gray')
    ax_dash1.set_title('Network Structure Overview', fontsize=18)
ax_dash1.axis('off')
stats_text = (f"Nodes: {num_nodes}, Edges: {num_edges}\nAvg. Degree: {avg_degree:.2f}, Density: {density:.4f}\n"
              f"Avg. Clustering: {avg_clustering_coeff:.4f}\nAvg. Path (LCC): {avg_path_length_lcc:.2f}" if not np.isnan(avg_path_length_lcc) else "Avg. Path (LCC): N/A")
ax_dash1.text(0.01, 0.01, stats_text, transform=ax_dash1.transAxes, fontsize=11,
              verticalalignment='bottom', bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.4))

# Panel 2: Centrality Highlights (e.g., Degree Distribution & Top Nodes)
ax_dash2 = axs_dashboard[0,1]
sns.histplot(degrees, bins=15, kde=True, ax=ax_dash2, color='mediumpurple', ec='black')
ax_dash2.set_title('Degree Distribution & Key Central Nodes', fontsize=18)
ax_dash2.set_xlabel('Degree', fontsize=14)
ax_dash2.set_ylabel('Frequency', fontsize=14)
top_deg_str = ", ".join([str(n) for n,c in top5_degree[:3]])
top_bet_str = ", ".join([str(n) for n,c in top5_betweenness[:3]])
central_nodes_text = (f"Top Degree: {top_deg_str}...\n"
                      f"Top Betweenness: {top_bet_str}...")
ax_dash2.text(0.98, 0.98, central_nodes_text, transform=ax_dash2.transAxes, fontsize=11,
              ha='right', va='top', bbox=dict(boxstyle='round,pad=0.5', fc='aliceblue', alpha=0.6))
ax_dash2.grid(True, linestyle=':', alpha=0.5)


    

ModuleNotFoundError: No module named 'seaborn'

ModuleNotFoundError: No module named 'seaborn'